In [2]:
import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.nn import DataParallel
from torchvision.io import read_image, ImageReadMode
from torchvision.datasets.vision import VisionDataset
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

import requests
import copy
import sys

from pathlib import Path
from PIL import Image
from imutils.paths import list_images, list_files
import os

from tqdm import tqdm

import matplotlib.patches as mpatches
import pandas as pd
import matplotlib as mpl

pv_vision_dir = os.path.join(Path.home(), 'pv-vision')
functions_dir = os.path.join(Path.home(), 'el_img_cracks_ec')

sys.path.append(pv_vision_dir)
sys.path.append(functions_dir)
import functions

from pv_vision.nn import ModelHandler
from tutorials.unet_model import construct_unet

Statistical analysis of class representation in our training set vs. LBNL's training set

In [3]:
my_map = {0 : 'empty',
          1 : 'busbar',
          2 : 'crack',
          3 : 'cross',
          4 : 'dark'}

In [4]:
# will put this method into util in the future
class SolarDataset(VisionDataset):
    """A dataset directly read images and masks from folder.    
    """
    def __init__(self, 
                 root, 
                 image_folder, 
                 mask_folder,
                 transforms,
                 mode = "train",
                 random_seed=42):
        super().__init__(root, transforms)
        self.image_path = Path(self.root) / image_folder
        self.mask_path = Path(self.root) / mask_folder

        if not os.path.exists(self.image_path):
            raise OSError(f"{self.image_path} not found.")

        if not os.path.exists(self.mask_path):
            raise OSError(f"{self.mask_path} not found.")

        self.image_list = sorted(list(list_images(self.image_path)))
        self.mask_list = sorted(list(list_images(self.mask_path)))

        self.image_list = np.array(self.image_list)
        self.mask_list = np.array(self.mask_list)

    def __len__(self):
        return len(self.image_list)

    def __getname__(self, index):
        image_name = os.path.splitext(os.path.split(self.image_list[index])[-1])[0]
        mask_name = os.path.splitext(os.path.split(self.mask_list[index])[-1])[0]

        if image_name == mask_name:
            return image_name
        else:
            return False
    
    def __getraw__(self, index):
        if not self.__getname__(index):
            raise ValueError("{}: Image doesn't match with mask".format(os.path.split(self.image_list[index])[-1]))
        image = Image.open(self.image_list[index])
        mask = Image.open(self.mask_list[index]).convert('L')
        mask = np.array(mask)
        mask = Image.fromarray(mask)

        return image, mask

    def __getitem__(self, index):
        image, mask = self.__getraw__(index)
        image, mask = self.transforms(image, mask)

        return image, mask

# will put into utils in the future
class Compose:
    def __init__(self, transforms):
        """
        transforms: a list of transform
        """
        self.transforms = transforms
    
    def __call__(self, image, target):
        """
        image: input image
        target: input mask
        """
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class FixResize:
    # UNet requires input size to be multiple of 16
    def __init__(self, size):
        self.size = size

    def __call__(self, image, target):
        image = F.resize(image, (self.size, self.size), interpolation=transforms.InterpolationMode.BILINEAR)
        target = F.resize(target, (self.size, self.size), interpolation=transforms.InterpolationMode.NEAREST)
        return image, target

class ToTensor:
    """Transform the image to tensor. Scale the image to [0,1] float32.
    Transform the mask to tensor.
    """
    def __call__(self, image, target):
        image = transforms.ToTensor()(image)
        target = torch.as_tensor(np.array(target), dtype=torch.int64)
        return image, target

class PILToTensor:
    """Transform the image to tensor. Keep raw type."""
    def __call__(self, image, target):
        image = F.pil_to_tensor(image)
        target = torch.as_tensor(np.array(target), dtype=torch.int64)
        return image, target

class Normalize:
    def __init__(self, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
        self.mean = mean
        self.std = std
    
    def __call__(self, image, target):
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image, target

Statistical analysis of class representation for LBNL's training set

In [5]:
root = Path('/home/eccoope/pv-vision/examples/crack_segmentation/img_label_for_training')
transformers = Compose([FixResize(256), ToTensor(), Normalize()])
trainset = SolarDataset(root, image_folder="train/img", 
        mask_folder="train/ann", transforms=transformers)

my_index = [trainset.__getname__(i) for i in range(len(trainset.image_list))]

df = pd.DataFrame(index=my_index, columns=[str(i) for i in range(5)])

for i in range(len(my_index)):
    name = trainset. __getname__(i)
    img, mask = trainset. __getitem__(i)
    mask_cpu = mask.cpu().numpy()

    vals, counts = np.unique(mask_cpu, return_counts=True)

    for v, c in zip(vals, counts):
        df.at[name, str(v)] = c

In [6]:
for i in range(5):
    counts = len(df[~df[str(i)].isna()])
    pct = counts/len(df)
    print(f'{np.round(100*pct,2)}% of images include Class {i} ({my_map[i]})')

100.0% of images include Class 0 (empty)
20.88% of images include Class 1 (busbar)
25.1% of images include Class 2 (crack)
39.2% of images include Class 3 (cross)
100.0% of images include Class 4 (dark)


In [7]:
for i in range(1, 4):
    for j in range(1, 4):
        
        if i == j:
            continue
        else:
            base = df[~df[str(i)].isna()]
            test = df[(~df[str(i)].isna()) & (~df[str(j)].isna())]
            pct = len(test)/len(base)
            print(f'{np.round(100*pct, 2)}% of images with Class {i} ({my_map[i]}) also contain Class {j} ({my_map[j]})')
    print('\n')

55.67% of images with Class 1 (busbar) also contain Class 2 (crack)
99.51% of images with Class 1 (busbar) also contain Class 3 (cross)


46.31% of images with Class 2 (crack) also contain Class 1 (busbar)
84.84% of images with Class 2 (crack) also contain Class 3 (cross)


53.02% of images with Class 3 (cross) also contain Class 1 (busbar)
54.33% of images with Class 3 (cross) also contain Class 2 (crack)




Statistical analysis of class representation for our training set

In [8]:
root = Path('/projects/wg-psel-ml/EL_images/eccoope')
transformers = functions.Compose([functions.FixResize(256), functions.ToTensor(), functions.Normalize()])
trainset = functions.SolarDataset(root, image_folder="img/train", 
        mask_folder="ann/train", transforms=transformers)
valset = functions.SolarDataset(root, image_folder="img/val", 
        mask_folder="ann/val", transforms=transformers)

In [9]:
my_index = [trainset.__getname__(i) for i in range(len(trainset.image_list))]

df2 = pd.DataFrame(index=my_index, columns=[str(i) for i in range(5)])

for i in range(len(my_index)):
    name = trainset. __getname__(i)
    img, mask = trainset. __getitem__(i)
    mask_cpu = mask.cpu().numpy()

    vals, counts = np.unique(mask_cpu, return_counts=True)

    for v, c in zip(vals, counts):
        df2.at[name, str(v)] = c

In [10]:
for i in range(5):
    counts = len(df2[~df2[str(i)].isna()])
    pct = counts/len(df2)
    print(f'{np.round(100*pct,2)}% of images include Class {i} ({my_map[i]})')

100.0% of images include Class 0 (empty)
100.0% of images include Class 1 (busbar)
43.8% of images include Class 2 (crack)
1.55% of images include Class 3 (cross)
45.35% of images include Class 4 (dark)


In [11]:
for i in range(1, 4):
    for j in range(1, 4):
        
        if i == j:
            continue
        else:
            base = df2[~df2[str(i)].isna()]
            test = df2[(~df2[str(i)].isna()) & (~df2[str(j)].isna())]
            pct = len(test)/len(base)
            print(f'{np.round(100*pct, 2)}% of images with Class {i} ({my_map[i]}) also contain Class {j} ({my_map[j]})')
    print('\n')

43.8% of images with Class 1 (busbar) also contain Class 2 (crack)
1.55% of images with Class 1 (busbar) also contain Class 3 (cross)


100.0% of images with Class 2 (crack) also contain Class 1 (busbar)
0.0% of images with Class 2 (crack) also contain Class 3 (cross)


100.0% of images with Class 3 (cross) also contain Class 1 (busbar)
0.0% of images with Class 3 (cross) also contain Class 2 (crack)


